In [3]:
import lmao
from lmao._utils import *

# ALICE Infologger Preprocessing

In [2]:
from lmao.preprocessing import LMAOPreprocessing

In [2]:
prep = LMAOPreprocessing(data_type='alice_info')

In [3]:
prep.from_parsed_csv(csv_path='dataset/ALICE-OSLdataset/phob_test_20220312_sample_struct_no_new15.csv')

'/Users/IsRealNPK/LMAO/dataset/ALICE-OSLdataset/phob_test_20220312_sample_struct_no_new15.csv'

In [4]:
prep.prep_data(label_path='dataset/OneDrive_1_2-5-2024/runs.csv',
               eor_path='dataset/OneDrive_1_2-5-2024/eor_reasons.csv')

Preprocessing to LMAO format with label from alice_info
Path :/Users/IsRealNPK/LMAO/dataset/OneDrive_1_2-5-2024/runs.csv


/Users/IsRealNPK/LMAO/lmao/preprocessing/alice_info.py:65: DtypeWarning: Columns (27,28,36) have mixed types. Specify dtype option on import or set low_memory=False.
  run_df = pd.read_csv(label_path)


Stating map RunID from label file


  0%|          | 0/23177 [00:00<?, ?it/s]

Stating map EOR from EOR file


  0%|          | 0/2831 [00:00<?, ?it/s]

Drop all severity D
Normalize EOR


  0%|          | 0/9712742 [00:00<?, ?it/s]

Getting Crashed from EOR
Returning dataframe
Done


In [63]:
prep.df

,session_id,severity,content,event_id,event_template,label,EOR,EOR_id,run_quality
134071,24064,I,writing 1 messages to aliecs.before_start_acti...,2,writing <*> messages to <*> (partition:<*>,not crashed,eor test.,61.0,test
134080,24064,I,overriding run start time in the GRP object to...,378,overriding run start time in the GRP object to...,not crashed,eor test.,61.0,test
134094,24064,I,writing 1 messages to aliecs.env_leave_state.C...,2,writing <*> messages to <*> (partition:<*>,not crashed,eor test.,61.0,test
134098,24064,I,starting new run,383,starting new run,not crashed,eor test.,61.0,test
135408,24064,I,Run number 526264,384,Run number <*>,not crashed,eor test.,61.0,test
...,...,...,...,...,...,...,...,...,...
34856755,24356,W,Non-contiguous timeframe IDs 187572 ... 187574,723,Non-contiguous timeframe IDs <*> ... <*>,crashed,triggers out synch. disabling ctp readout.,59.0,bad
34856756,24356,W,Non-contiguous timeframe IDs 187572 ... 187574,723,Non-contiguous timeframe IDs <*> ... <*>,crashed,triggers out synch. disabling ctp readout.,59.0,bad
34856757,24356,W,Non-contiguous timeframe IDs 187572 ... 187574,723,Non-contiguous timeframe IDs <*> ... <*>,crashed,triggers out synch. disabling ctp readout.,59.0,bad
34856758,24356,W,Gap in timeframe ids detected: previous= 18757...,724,Gap in timeframe ids <*>,crashed,triggers out synch. disabling ctp readout.,59.0,bad


In [66]:
prep.df.to_parquet('dataset/ALICE-OSLdataset/ALICE_info_no_severity_D_with_label.parquet.gzip', compression='gzip')

# ALICE Bertopic

In [5]:
from lmao.topicmodel import LMAOTopic
topic_model = LMAOTopic()

Topic model mode set to :BERTopic.
Start Initialized


In [8]:
df = pd.read_parquet('dataset/ALICE-OSLdataset/ALICE_info_no_severity_D_with_label.parquet.gzip')

In [ ]:
topic_model.training_model(df=df)

In [ ]:
topic_dict = topic_model.topic_dict

In [11]:
%%timeit
topic_model.model.transform(df.content.tolist()[:1000])
# topic, _ = topic_model.model.transform(df.content.tolist()[0])
# topic

1.25 s ± 304 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
topic_model.from_trained_model(model_path='lmao/topicmodel/model/BERTopic-ALICE_info_no_severity_D-all-MiniLM-L6-v2')

Done


In [7]:
topic_dict = topic_model.topic_dict

In [12]:
df = pd.read_parquet('dataset/ALICE-OSLdataset/ALICE_info_no_severity_D_with_label.parquet.gzip')

In [13]:
df

,session_id,severity,content,event_id,event_template,label,EOR,EOR_id,run_quality
134071,24064,I,writing 1 messages to aliecs.before_start_acti...,2,writing <*> messages to <*> (partition:<*>,not crashed,eor test.,61.0,test
134080,24064,I,overriding run start time in the GRP object to...,378,overriding run start time in the GRP object to...,not crashed,eor test.,61.0,test
134094,24064,I,writing 1 messages to aliecs.env_leave_state.C...,2,writing <*> messages to <*> (partition:<*>,not crashed,eor test.,61.0,test
134098,24064,I,starting new run,383,starting new run,not crashed,eor test.,61.0,test
135408,24064,I,Run number 526264,384,Run number <*>,not crashed,eor test.,61.0,test
...,...,...,...,...,...,...,...,...,...
34856755,24356,W,Non-contiguous timeframe IDs 187572 ... 187574,723,Non-contiguous timeframe IDs <*> ... <*>,crashed,triggers out synch. disabling ctp readout.,59.0,bad
34856756,24356,W,Non-contiguous timeframe IDs 187572 ... 187574,723,Non-contiguous timeframe IDs <*> ... <*>,crashed,triggers out synch. disabling ctp readout.,59.0,bad
34856757,24356,W,Non-contiguous timeframe IDs 187572 ... 187574,723,Non-contiguous timeframe IDs <*> ... <*>,crashed,triggers out synch. disabling ctp readout.,59.0,bad
34856758,24356,W,Gap in timeframe ids detected: previous= 18757...,724,Gap in timeframe ids <*>,crashed,triggers out synch. disabling ctp readout.,59.0,bad


In [14]:
topic_model.get_topic(df=df)

,session_id,severity,content,event_id,event_template,label,EOR,EOR_id,run_quality,topic
134071,24064,I,writing 1 messages to aliecs.before_start_acti...,2,writing <*> messages to <*> (partition:<*>,not crashed,eor test.,61.0,test,572
134080,24064,I,overriding run start time in the GRP object to...,378,overriding run start time in the GRP object to...,not crashed,eor test.,61.0,test,1682
134094,24064,I,writing 1 messages to aliecs.env_leave_state.C...,2,writing <*> messages to <*> (partition:<*>,not crashed,eor test.,61.0,test,464
134098,24064,I,starting new run,383,starting new run,not crashed,eor test.,61.0,test,-1
135408,24064,I,Run number 526264,384,Run number <*>,not crashed,eor test.,61.0,test,1635
...,...,...,...,...,...,...,...,...,...,...
34856755,24356,W,Non-contiguous timeframe IDs 187572 ... 187574,723,Non-contiguous timeframe IDs <*> ... <*>,crashed,triggers out synch. disabling ctp readout.,59.0,bad,1882
34856756,24356,W,Non-contiguous timeframe IDs 187572 ... 187574,723,Non-contiguous timeframe IDs <*> ... <*>,crashed,triggers out synch. disabling ctp readout.,59.0,bad,1882
34856757,24356,W,Non-contiguous timeframe IDs 187572 ... 187574,723,Non-contiguous timeframe IDs <*> ... <*>,crashed,triggers out synch. disabling ctp readout.,59.0,bad,1882
34856758,24356,W,Gap in timeframe ids detected: previous= 18757...,724,Gap in timeframe ids <*>,crashed,triggers out synch. disabling ctp readout.,59.0,bad,-1


In [15]:
df.to_parquet('dataset/ALICE-OSLdataset/ALICE_info_no_severity_D_with_topic.parquet.gzip')

# LMAO graph

In [8]:
from lmao.graph import Graph

In [13]:
data = Graph(df=pd.read_parquet('dataset/ALICE-OSLdataset/ALICE_info_no_severity_D_with_topic.parquet.gzip'),mode='pyg', padding=False)

Topics size of this dataframe is : 2149
Getting Group by from dataframe


  0%|          | 0/126 [00:00<?, ?it/s]

Max sequence size of this dataframe is : 772369
Please call .get_PyG() to get PyG dataloader


In [10]:
G = data.get_one_graph(0, no_feature=True)

In [11]:
from lmao.visualizer.plot import plotly_digraph

In [12]:
plotly_digraph(G, label='y', topic_dict=topic_dict)